In [0]:
spark.sql("USE CATALOG mvp")
spark.sql("USE SCHEMA gold")

In [0]:
df_amostra_gold = spark.table("mvp.gold.dados_postos_anp_g").limit(100)
display(df_amostra_gold)

In [0]:
query = """
SELECT 
  COUNT(DISTINCT CNPJ) AS num_cnpjs_distintos,
  COUNT(DISTINCT Num_reg_anp) AS num_reg_anp_distintos,
  COUNT(DISTINCT C_digo_Instala__o_i_Simp) AS num_instalacoes_distintas
FROM mvp.gold.dados_postos_anp_g
"""
df_contagem_distintos = spark.sql(query)
display(df_contagem_distintos)

In [0]:
# Contagem total de postos de combustíveis no Brasil pelo número de instalações distintas
query = """
SELECT COUNT(DISTINCT C_digo_Instala__o_i_Simp) AS num_instalacoes_distintas_brasil
FROM mvp.gold.dados_postos_anp_g
"""
df_total_postos_brasil = spark.sql(query)
display(df_total_postos_brasil)

In [0]:
# Contagem de postos de combustíveis por UF, distinguindo pelo número de instalações distintas
query = """
SELECT 
  UF,
  COUNT(DISTINCT C_digo_Instala__o_i_Simp) AS num_instalacoes_distintas
FROM mvp.gold.dados_postos_anp_g
GROUP BY UF
ORDER BY num_instalacoes_distintas DESC
"""
df_postos_por_uf = spark.sql(query)
display(df_postos_por_uf)

In [0]:
# Contagem de postos por distribuidor, distinguindo pelo número da instalação
# Inclui coluna de percentual de participação e totalizador ao final
query = """
WITH total_instalacoes AS (
  SELECT COUNT(DISTINCT C_digo_Instala__o_i_Simp) AS total
  FROM mvp.gold.dados_postos_anp_g
),
instalacoes_por_distribuidor AS (
  SELECT 
    Vincula__o_a_Distribuidor,
    COUNT(DISTINCT C_digo_Instala__o_i_Simp) AS NUM_POSTOS
  FROM mvp.gold.dados_postos_anp_g
  GROUP BY Vincula__o_a_Distribuidor
),
ranking AS (
  SELECT 
    i.Vincula__o_a_Distribuidor,
    i.NUM_POSTOS,
    ROUND(100.0 * i.NUM_POSTOS / t.total, 2) AS PERCENTUAL
  FROM instalacoes_por_distribuidor i
  CROSS JOIN total_instalacoes t
)
SELECT 
  Vincula__o_a_Distribuidor,
  NUM_POSTOS,
  CONCAT(PERCENTUAL, '%') AS PERCENTUAL
FROM ranking
UNION ALL
SELECT 
  'TOTAL',
  SUM(NUM_POSTOS),
  CONCAT(ROUND(SUM(PERCENTUAL), 2), '%')
FROM ranking
ORDER BY 
  CASE WHEN Vincula__o_a_Distribuidor = 'TOTAL' THEN 1 ELSE 0 END,
  NUM_POSTOS DESC
"""
df_postos_por_distribuidorf = spark.sql(query)
display(df_postos_por_distribuidorf)

In [0]:
# Contagem de postos de combustíveis Vincula__o_a_Distribuidor SEJA "BANDEIRA BRANCA" por UF,
# distinguindo pelo número de instalações distintas, incluir comentários e o percentual de representatividade por UF.
# Ao final colocar um totalizador

query = """
WITH total_instalacoes AS (
  SELECT COUNT(DISTINCT C_digo_Instala__o_i_Simp) AS total
  FROM mvp.gold.dados_postos_anp_g
  WHERE Vincula__o_a_Distribuidor = 'BANDEIRA BRANCA'
),
instalacoes_por_uf AS (
  SELECT 
    UF,
    COUNT(DISTINCT C_digo_Instala__o_i_Simp) AS num_instalacoes_distintas
  FROM mvp.gold.dados_postos_anp_g
  WHERE Vincula__o_a_Distribuidor = 'BANDEIRA BRANCA'
  GROUP BY UF
),
ranking AS (
  SELECT 
    i.UF,
    i.num_instalacoes_distintas,
    ROUND(100.0 * i.num_instalacoes_distintas / t.total, 2) AS percentual
  FROM instalacoes_por_uf i
  CROSS JOIN total_instalacoes t
)
SELECT 
  UF,
  num_instalacoes_distintas,
  CONCAT(percentual, '%') AS percentual
FROM ranking
UNION ALL
SELECT 
  'TOTAL',
  SUM(num_instalacoes_distintas),
  CONCAT(ROUND(SUM(percentual), 2), '%')
FROM ranking
ORDER BY 
  CASE WHEN UF = 'TOTAL' THEN 1 ELSE 0 END,
  num_instalacoes_distintas DESC
"""
df_postos_bandeira_branca_por_uf = spark.sql(query)
display(df_postos_bandeira_branca_por_uf)